<a href="https://colab.research.google.com/github/Balaji-731/AI_Internship/blob/main/Shork_Vosk_vs_Whisper_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Shork Vosk vs Whisper Script
!pip install -q vosk gTTS faster-whisper pydub soundfile sentencepiece
!apt -qq install -y ffmpeg

from google.colab import files
from gtts import gTTS
from IPython.display import Audio
import os, wave, json, subprocess
## Audio Prep
print("Upload audio (optional). Skip to auto create tts sample.")
up=files.upload()

if up:
  audio=list(up.keys())[0]
else:
  tts=gTTS("Hello! This is a test audio for comparing Vosk and Whisper.", lang="en")
  audio="sample.mp3"
  tts.save(audio)
## convert->WAV 16khz mono
wav="audio.wav"
subprocess.run(["ffmpeg", "-y","-i", audio, "-ar", "16000", "-ac", "1", wav],
               stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
Audio(wav)

## vosk
if not os.path.exists("vosk-model"):
  print("Downloading Vosk model...")
  !rm -f model.zip # Ensure a clean slate
  !wget https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip -O model.zip
  !unzip model.zip
  !mv vosk-model-en-us-0.22 vosk-model
  !rm model.zip

from vosk import Model, KaldiRecognizer
wf=wave.open(wav, "rb")
rec=KaldiRecognizer(Model("vosk-model"), wf.getframerate())
vosk_text=""

while True:
  data=wf.readframes(4000)
  if not data: break
  if rec.AcceptWaveform(data):
    vosk_text+=json.loads(rec.Result()).get("text"," ")+" "
vosk_text+=json.loads(rec.FinalResult()).get("text"," ")

print("\n>>> VOSK:\n",vosk_text.strip())

## whisper

import torch
from faster_whisper import WhisperModel
device="cuda" if torch.cuda.is_available() else "cpu"
wmodel=WhisperModel("small", device=device)

segments,_=wmodel.transcribe(wav)
whisper_text=" ".join([s.text for s in segments]).strip()

print("\n>>> WHISPER:\n",whisper_text)